In [1]:
import io
import itertools

import numpy as np
import sklearn.metrics

import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

import matplotlib.pyplot as plt

In [2]:
# Loading the datasets
data_train = np.load(r"Dataset/Trousers & Jeans - All - Train.npz")
data_val = np.load(r"Dataset/Trousers & Jeans - All - Validation.npz")
data_test = np.load(r"Dataset/Trousers & Jeans - All - Test.npz")

In [3]:
# Extracting the arrays from the imported data
images_train = data_train['images']
labels_train = data_train['labels']

images_val = data_val['images']
labels_val = data_val['labels']

images_test = data_test['images']
labels_test = data_test['labels']

In [4]:
# Scaling the pixel values of all images 
images_train = images_train/255.0
images_val = images_val/255.0
images_test = images_test/255.0

In [5]:
# Defining the hyperparameters we would tune, and their values to be tested
HP_FILTER_SIZE = hp.HParam('filter_size', hp.Discrete([3,5,7]))
HP_FILTER_NUM = hp.HParam('filters_number', hp.Discrete([32,64,96,128]))

METRIC_ACCURACY = 'accuracy'

# Logging setup info
with tf.summary.create_file_writer(r'Logs/Model 1/hparam_tuning/').as_default():
    hp.hparams_config(
        hparams=[HP_FILTER_SIZE, HP_FILTER_NUM],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
    )

In [6]:
EPOCHS = 20
BATCH_SIZE = 64

In [7]:
# Wrapping our model and training in a function
def train_test_model(hparams, session_num):
    
    # Outlining the model/architecture of our CNN
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(hparams[HP_FILTER_NUM], hparams[HP_FILTER_SIZE], activation='relu', input_shape=(120,90,3)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(4)
    ])
    
    # Defining the loss function
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

    # Compiling the model
    model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])

    # Defining the logging directory
    log_dir = "Logs/Model 1/fit/" + "run-{}".format(session_num)
    
    def plot_confusion_matrix(cm, class_names):
        """
        Returns a matplotlib figure containing the plotted confusion matrix.

        Args:
          cm (array, shape = [n, n]): a confusion matrix of integer classes
          class_names (array, shape = [n]): String names of the integer classes
        """
        figure = plt.figure(figsize=(12, 12))
        plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
        plt.title("Confusion matrix")
        plt.colorbar()
        tick_marks = np.arange(len(class_names))
        plt.xticks(tick_marks, class_names, rotation=45)
        plt.yticks(tick_marks, class_names)

        # Normalize the confusion matrix.
        cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

        # Use white text if squares are dark; otherwise black.
        threshold = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            color = "white" if cm[i, j] > threshold else "black"
            plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')
        return figure
    
     
    
    
    def plot_to_image(figure):
        """Converts the matplotlib plot specified by 'figure' to a PNG image and
        returns it. The supplied figure is closed and inaccessible after this call."""
        # Save the plot to a PNG in memory.
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        # Closing the figure prevents it from being displayed directly inside
        # the notebook.
        plt.close(figure)
        buf.seek(0)
        # Convert PNG buffer to TF image
        image = tf.image.decode_png(buf.getvalue(), channels=4)
        # Add the batch dimension
        image = tf.expand_dims(image, 0)
        return image
    
    
    # Defining a file writer for Confusion Matrix logging purposes
    file_writer_cm = tf.summary.create_file_writer(log_dir + '/cm') 
    
    
    def log_confusion_matrix(epoch, logs):
        # Use the model to predict the values from the validation dataset.
        test_pred_raw = model.predict(images_val)
        test_pred = np.argmax(test_pred_raw, axis=1)

        # Calculate the confusion matrix.
        cm = sklearn.metrics.confusion_matrix(labels_val, test_pred)
        # Log the confusion matrix as an image summary.
        figure = plot_confusion_matrix(cm, class_names=['Trousers Male', 'Jeans Male', 'Trousers Female', "Jeans Female"])
        cm_image = plot_to_image(figure)

        # Log the confusion matrix as an image summary.
        with file_writer_cm.as_default():
            tf.summary.image("Confusion Matrix", cm_image, step=epoch)
    
    
    # Define the Tensorboard and Confusion Matrix callbacks.
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, profile_batch=0)
    cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

    
    # Defining early stopping to prevent overfitting
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor = 'val_loss',
        mode = 'auto',
        min_delta = 0,
        patience = 2,
        verbose = 0, 
        restore_best_weights = True
    )
    
    # Training the model
    model.fit(
        images_train,
        labels_train,
        epochs = EPOCHS,
        batch_size = BATCH_SIZE,
        callbacks = [tensorboard_callback, cm_callback, early_stopping],
        validation_data = (images_val,labels_val),
        verbose = 2
    )
    
    # Evaluating the model's performance on the validation set
    _, accuracy = model.evaluate(images_val,labels_val)
    
    # Saving the current model for future reference
    model.save(r"saved_models/Model 1/Run-{}".format(session_num))
    
    return accuracy

In [8]:
# Creating a function to log the resuls
def run(log_dir, hparams, session_num):
    
    with tf.summary.create_file_writer(log_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        accuracy = train_test_model(hparams, session_num)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [9]:
session_num = 1

for filter_size in HP_FILTER_SIZE.domain.values:
    for filter_num in HP_FILTER_NUM.domain.values:

        hparams = {
            HP_FILTER_SIZE: filter_size,
            HP_FILTER_NUM: filter_num
        }

        run_name = "run-%d" % session_num
        print('--- Starting trial: %s' % run_name)
        print({h.name: hparams[h] for h in hparams})
        run('Logs/Model 1/hparam_tuning/' + run_name, hparams, session_num)

        session_num += 1

--- Starting trial: run-1
{'filter_size': 3, 'filters_number': 32}
Epoch 1/20
63/63 - 18s - loss: 6.4006 - accuracy: 0.4291 - val_loss: 1.0554 - val_accuracy: 0.5720
Epoch 2/20
63/63 - 15s - loss: 0.8807 - accuracy: 0.6451 - val_loss: 0.9674 - val_accuracy: 0.6060
Epoch 3/20
63/63 - 13s - loss: 0.7163 - accuracy: 0.7203 - val_loss: 0.8793 - val_accuracy: 0.6360
Epoch 4/20
63/63 - 14s - loss: 0.6380 - accuracy: 0.7545 - val_loss: 0.8512 - val_accuracy: 0.6600
Epoch 5/20
63/63 - 16s - loss: 0.5794 - accuracy: 0.7737 - val_loss: 0.8095 - val_accuracy: 0.6740
Epoch 6/20
63/63 - 17s - loss: 0.5282 - accuracy: 0.8002 - val_loss: 0.8241 - val_accuracy: 0.6940
Epoch 7/20
63/63 - 15s - loss: 0.4700 - accuracy: 0.8284 - val_loss: 0.8158 - val_accuracy: 0.6720
16/16 [==============================] - 1s 47ms/step - loss: 0.8095 - accuracy: 0.6740
INFO:tensorflow:Assets written to: saved_models/Model 1/Run-1/assets
--- Starting trial: run-2
{'filter_size': 3, 'filters_number': 64}
Epoch 1/20
63/63

16/16 [==============================] - 1s 57ms/step - loss: 0.9506 - accuracy: 0.6180
INFO:tensorflow:Assets written to: saved_models/Model 1/Run-9/assets
--- Starting trial: run-10
{'filter_size': 7, 'filters_number': 64}
Epoch 1/20
63/63 - 60s - loss: 3.3039 - accuracy: 0.4933 - val_loss: 1.1070 - val_accuracy: 0.5320
Epoch 2/20
63/63 - 50s - loss: 0.9586 - accuracy: 0.6139 - val_loss: 0.9522 - val_accuracy: 0.6180
Epoch 3/20
63/63 - 46s - loss: 0.8653 - accuracy: 0.6648 - val_loss: 0.9320 - val_accuracy: 0.6300
Epoch 4/20
63/63 - 43s - loss: 0.7929 - accuracy: 0.6963 - val_loss: 0.8898 - val_accuracy: 0.6700
Epoch 5/20
63/63 - 42s - loss: 0.7637 - accuracy: 0.6978 - val_loss: 0.9139 - val_accuracy: 0.6620
Epoch 6/20
63/63 - 42s - loss: 0.6924 - accuracy: 0.7213 - val_loss: 0.9049 - val_accuracy: 0.6560
16/16 [==============================] - 1s 86ms/step - loss: 0.8898 - accuracy: 0.6700
INFO:tensorflow:Assets written to: saved_models/Model 1/Run-10/assets
--- Starting trial: run

In [10]:
# Loading a model to evaluate on the test set
model = tf.keras.models.load_model(r"saved_models/Model 1/Run-1")

In [11]:
test_loss, test_accuracy = model.evaluate(images_test,labels_test)

16/16 [==============================] - 1s 37ms/step - loss: 0.8125 - accuracy: 0.6980


In [12]:
# Printing the test results
print('Test loss: {0:.4f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.8125. Test accuracy: 69.80%


In [ ]:
%load_ext tensorboard
%tensorboard --logdir "Logs/Model 1/hparam_tuning"

In [ ]:
%load_ext tensorboard
%tensorboard --logdir "Logs/Model 1/fit"